# 1. Coleta de dados

### API Yahoo Finance

In [ ]:
# intalar e/ou atualizar biblioteca yfinance
!pip install yfinance --upgrade

### Dados ações BOVA11 e S&P500

In [1]:
# importar yfinance ao notebook
import yfinance as yf

# variávis de definição - dados de interesse
ticker_label = ["BOVA11.SA", "^GSPC"]
dt_period = ["2009-01-01", "2021-12-31"]
historical_data = {}

# recuperar dados de interesse
for label in ticker_label:
    data = yf.Ticker(label)
    historical_data[label] = data.history(start=dt_period[0], end=dt_period[1])
    historical_data[label].reset_index(inplace=True)
    historical_data[label].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3223 entries, 0 to 3222
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          3223 non-null   datetime64[ns]
 1   Open          3223 non-null   float64       
 2   High          3223 non-null   float64       
 3   Low           3223 non-null   float64       
 4   Close         3223 non-null   float64       
 5   Volume        3223 non-null   int64         
 6   Dividends     3223 non-null   int64         
 7   Stock Splits  3223 non-null   int64         
dtypes: datetime64[ns](1), float64(4), int64(3)
memory usage: 201.6 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3272 entries, 0 to 3271
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          3272 non-null   datetime64[ns]
 1   Open          3272 non-null   float64       
 2   High        

### Dados cotação dólar americano

In [2]:
"""
Download realizado através do site investing.com
https://br.investing.com/currencies/usd-brl-historical-data
"""

# importar pandas ao notebook
import pandas as pd

# ler arquivo csv
dolar = pd.read_csv("USD_BRL Dados Históricos.csv", parse_dates=['Data'], dayfirst=True)

dolar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Data      3387 non-null   datetime64[ns]
 1   Último    3387 non-null   float64       
 2   Abertura  3387 non-null   float64       
 3   Máxima    3387 non-null   float64       
 4   Mínima    3387 non-null   float64       
 5   Var%      3387 non-null   float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 158.9 KB


# 2. Preocessamento dos dados

### Junção dos conjuntos de dados

In [3]:
# Colunas de interesse
colunas_yf = ['Date', 'Close']
colunas_dolar = ['Data', 'Último']

# Datasets individuais
bova = historical_data['BOVA11.SA'][colunas_yf]
bova.columns = ['data', 'BOVA']

sp5 = historical_data['^GSPC'][colunas_yf]
sp5.columns = ['data', 'SP5']

usd = dolar[colunas_dolar]
usd.columns = ['data', 'USD']

# Jução dos datasets
historico = bova.set_index('data').join(usd.set_index('data'))
historico = historico.join(sp5.set_index('data'))

# Verificação de valores nulos
historico.isnull().sum()

BOVA     0
USD      0
SP5     79
dtype: int64

### Investigação e tratamento de valores nulos S&P500

In [4]:
nulos = historico[historico.SP5.isnull()]
print(nulos.head(),
      "\n",
      nulos.tail())

                 BOVA     USD  SP5
data                              
2009-01-19  38.950001  2.3545  NaN
2009-02-16  41.990002  2.2755  NaN
2009-05-25  50.900002  2.0265  NaN
2009-07-03  50.799999  1.9494  NaN
2009-11-26  65.500000  1.7475  NaN 
                   BOVA     USD  SP5
data                               
2021-01-18  116.540001  5.2957  NaN
2021-05-31  121.250000  5.2172  NaN
2021-07-05  121.970001  5.0910  NaN
2021-09-06  113.480003  5.1693  NaN
2021-11-25  101.989998  5.5664  NaN


In [5]:
import numpy as np

# Substituir valores nulos
for i in range(len(historico)):
    if (np.isnan(historico.iloc[i,2])):
        historico.iloc[i,2] = historico.iloc[i-1,2]

# Verificação de valores nulos
historico.isnull().sum()

BOVA    0
USD     0
SP5     0
dtype: int64

In [6]:
# Conjunto tratado
historico.describe()

,BOVA,USD,SP5
count,3223.000000,3223.000000,3223.000000
mean,68.560187,3.081228,2158.809627
std,21.013876,1.217166,923.220577
min,36.450001,1.538300,676.530029
25%,52.880001,2.012750,1351.859985
50%,62.200001,3.118500,2051.120117
75%,81.000000,3.853800,2748.864990
max,125.750000,5.885600,4793.060059


### Feature Engineering

#### Cálculo de retorno 

In [7]:
import numpy as np

dados = historico.copy()

# Cálculo de retoro diário
col = dados.columns
for i in col[0:4]:
    dados['RT_'+str(i)] = (dados[str(i)] /dados[str(i)].shift(periods=1)) - 1

# Remover valores nulos
dataset =  dados.iloc[:,3:].copy()
dataset.dropna(subset = ["RT_BOVA"], inplace=True)

#### Variável alvo

In [8]:
# Retorno mínimo esperado
retorno_min = 0.005

# Variável alvo
def target(vetor):
    rt = vetor['RT_BOVA']
    if rt>=retorno_min:
        return 1
    else:
        return 0

# Criar alvo, deslocamento temporal e remover nulos 
dataset['TARGET'] = dataset.apply(target, axis=1)
dataset['TARGET'] = dataset['TARGET'].shift(-1)
dataset.dropna(subset = ["TARGET"], inplace=True)

dataset.head()

,RT_BOVA,RT_USD,RT_SP5,TARGET
data,,,,
2009-01-05,0.041667,-0.036822,-0.004668,1.0
2009-01-06,0.007619,-0.029958,0.007817,0.0
2009-01-07,-0.039698,0.047476,-0.030010,1.0
2009-01-08,0.038386,0.010121,0.003397,0.0
2009-01-09,-0.004739,-0.018994,-0.021303,0.0


In [9]:
dataset.describe()

,RT_BOVA,RT_USD,RT_SP5,TARGET
count,3221.000000,3221.000000,3221.000000,3221.000000
mean,0.000416,0.000332,0.000574,0.357032
std,0.016274,0.010289,0.011410,0.479199
min,-0.145747,-0.057731,-0.119841,0.000000
25%,-0.008155,-0.005621,-0.003480,0.000000
50%,0.000182,0.000318,0.000598,0.000000
75%,0.009200,0.006242,0.005647,1.000000
max,0.133951,0.074251,0.093828,1.000000


#### Salvar datasets

In [10]:
dados.iloc[:,0:3].to_excel("serie_hist.xlsx")
dataset.to_excel("retorno_hist.xlsx")